In [1]:
import pandas as pd
import json

PLAYTIME_THRESHOLD = 10

def convert_game(game: object) -> pd.Series:
    game = pd.json_normalize(game)
    game["playtime_forever"] = game["playtime_forever"].apply(lambda p: p / 60)
    game = game.where(lambda p: p["playtime_forever"] > PLAYTIME_THRESHOLD).dropna()
    return game

games = pd.read_json("games.json")
games["games"] = games["games"].apply(convert_game)
games["games"][0]

,appid,playtime_forever
3,218620.0,10.800000
10,730.0,688.800000
13,470220.0,16.400000
15,360430.0,33.183333
16,289070.0,28.483333
20,252950.0,2871.683333
22,578080.0,199.066667
25,695290.0,136.433333
41,1089350.0,64.366667


In [2]:
def convert_game_info(gi):
    gi["appid"] = gi["game_info"]["steam_appid"]
    gi["name"] = gi["game_info"]["name"]
    if gi["game_info"]["genres"] is None:
        gi["game_info"]["genres"] = []
    gi["genres"] = list(map(lambda g: g["description"], gi["game_info"]["genres"]))
    if len(gi["reviews"]) > 0:
        gi["reviews"] = round(float(gi["reviews"][0]["weighted_vote_score"]), ndigits=2)
    else:
        return None
    return gi

game_info = pd.read_json("game_info.json")
game_info = game_info.apply(convert_game_info, axis=1)
game_info

,game_info,reviews,review_summary,appid,name,genres
0,"{'name': 'Warframe', 'steam_appid': 230410, 'g...",0.52,"{'review_score': 8.0, 'total_positive': 1009, ...",230410.0,Warframe,"[Action, Free to Play]"
1,"{'name': 'Rust', 'steam_appid': 252490, 'genre...",0.91,"{'review_score': 8.0, 'total_positive': 169918...",252490.0,Rust,"[Action, Adventure, Indie, Massively Multiplay..."
2,"{'name': 'Grand Theft Auto: San Andreas', 'ste...",0.52,"{'review_score': 7.0, 'total_positive': 39, 't...",12250.0,Grand Theft Auto: San Andreas,[]
3,"{'name': 'Grand Theft Auto: San Andreas', 'ste...",0.77,"{'review_score': 8.0, 'total_positive': 12549,...",12120.0,Grand Theft Auto: San Andreas,[Action]
4,"{'name': 'Game Dev Tycoon', 'steam_appid': 239...",0.70,"{'review_score': 8.0, 'total_positive': 16966,...",239820.0,Game Dev Tycoon,"[Casual, Indie, Simulation, Strategy]"
...,...,...,...,...,...,...
15821,"{'name': 'The Wind Road 紫塞秋风', 'steam_appid': ...",0.55,"{'review_score': 8.0, 'total_positive': 52, 't...",1264670.0,The Wind Road 紫塞秋风,"[Action, Adventure, Indie, RPG]"
15822,{'name': 'Stifled - Echolocation Horror Myster...,0.59,"{'review_score': 6.0, 'total_positive': 14, 't...",514830.0,Stifled - Echolocation Horror Mystery,"[Action, Adventure, Indie, Simulation]"
15823,"{'name': 'Shoot 1UP', 'steam_appid': 373610, '...",0.62,"{'review_score': 7.0, 'total_positive': 41, 't...",373610.0,Shoot 1UP,"[Action, Indie]"
15824,{'name': 'State of War : Warmonger / 蓝色警戒 (Cla...,0.54,"{'review_score': 8.0, 'total_positive': 51, 't...",748040.0,State of War : Warmonger / 蓝色警戒 (Classic 2000),"[Action, Strategy]"


In [3]:
def convert_to_percentage(game_time):
    total_playtime = sum(game_time)
    playtime_percentages = []
    for playtime in game_time:
        playtime_to_percent = (playtime / total_playtime) * 100
        playtime_percentages.append(playtime_to_percent)
    return playtime_percentages;

for i in range(len(games["games"])):
    games["games"][i]["playtime_forever"] = convert_to_percentage(games["games"][i]["playtime_forever"])

In [8]:
all_genres = []
for app_genre_list in game_info['genres']:
    if app_genre_list is not None:
        for genre in app_genre_list:
            all_genres.append(genre)
unique_genres = set(all_genres)
print(unique_genres)

{'Video Production', 'Free to Play', 'Massively Multiplayer', 'Animation & Modeling', 'Web Publishing', 'Violent', 'Strategy', 'Nudity', 'Early Access', 'Racing', 'Movie', 'Short', 'RPG', 'Education', 'Utilities', 'Accounting', 'Adventure', 'Simulation', 'Design & Illustration', 'Episodic', 'Game Development', 'Action', 'Audio Production', 'Photo Editing', 'Software Training', 'Documentary', 'Sexual Content', 'Gore', 'Sports', 'Casual', 'Indie', 'Tutorial'}
